In [7]:
### ORIGINAL SOURCE : https://github.com/philk27/checkOpenAiApiKey.git

import openai
import os
import sys
import requests

In [8]:
API_KEY = "##___OPENAI_API_KEY___##"

HARDCODED_MODELS = [
    "text-davinci-002",
    "code-davinci-002",
    "text-davinci-003",
    "gpt-3.5-turbo-0301",
    "gpt-3.5-turbo-16k",
    "gpt-3.5-turbo-0613",
    "gpt-3.5-turbo-16k-0613",
    "gpt-3.5-turbo",
    "gpt-4",
    "gpt-4-0314",
    "gpt-4-32k",
    "gpt-4-32k-0314"
]

In [9]:
def connectToOpenAI(apiKey):
    try:
        openai.api_key = apiKey
        response = requests.get("https://api.openai.com")
        if response.text.strip():
            print("Successfully connected to OpenAI API.")
        else:
            print("Failed to connect to OpenAI. Empty response received.")
            sys.exit(1)

        if "Retry-After" in response.headers:
            retry_after = int(response.headers["Retry-After"])
            print(f"API rate limit exceeded. Retry after {retry_after} seconds.")
            sys.exit(1)
        models = listModels()

    except requests.exceptions.RequestException as e:
        print(f"Failed to connect to OpenAI: {str(e)}")
        sys.exit(1)
    except openai.error.AuthenticationError:
        print("Authentication failed. Please check your API key.")
        sys.exit(1)
    except openai.error.AuthorizationError:
        print("Authorization failed. Please check your permissions.")
        sys.exit(1)

def listModels():
    try:
        models = openai.Model.list()
        availableModels = sorted([model.id for model in models["data"]])
        for model in availableModels:
            print(f"- {model}")
        return availableModels
    except openai.error.AuthenticationError:
        print("Authentication failed. Please check your API key.")
        sys.exit(1)
    except openai.error.AuthorizationError:
        print("Authorization failed. Please check your permissions.")
        sys.exit(1)
    except openai.error.RateLimitError as e:
        retry_after = e.response.headers.get("Retry-After")
        if retry_after:
            print(f"API rate limit exceeded. Retry after {retry_after} seconds.")
        else:
            print("API rate limit exceeded.")
        sys.exit(1)
    except Exception as e:
        print(f"Failed to list models: {e}")
        sys.exit(1)

def checkMissingModels(availableModels):
    requiredModels = []
    if os.path.isfile("oaimodellist.txt"):
        with open("oaimodellist.txt", "r") as file:
            requiredModels = [line.strip() for line in file if line.strip()]
    else:
        requiredModels = HARDCODED_MODELS

    missingModels = [model for model in requiredModels if model not in availableModels]

    if missingModels:
        print("\nMissing models:")
        for model in missingModels:
            print(f"- {model}")
    else:
        print("\nAccess to all required models.")

def listFiles(apiKey):
    try:
        url = "https://api.openai.com/v1/files"
        headers = {"Authorization": f"Bearer {apiKey}"}
        response = requests.get(url, headers=headers)
        files = response.json()["data"]

        print("\nFiles available to you:")
        if len(files) == 0:
            print("None")
        else:
            for file in files:
                print(f"- {file['id']}")

        filesDebug = response.json()
    except Exception as e:
        print(f"Failed to list files: {e}")

In [10]:
connectToOpenAI(API_KEY)
availableModels = listModels()
checkMissingModels(availableModels)
listFiles(API_KEY)

Successfully connected to OpenAI API.
Authentication failed. Please check your API key.


AttributeError: module 'openai.error' has no attribute 'AuthorizationError'